## Proyecto: Campañas de Marketing directo para depósitos bancarios a plazo fijo

## Descripción del Proyecto y Origen del Dataset

El dataset corresponde a una recopilación de datos relacionados a campañas de marketing, realizados mediante llamadas telefónicas a los clientes.

El objetivo es evaluar la aceptación de un depósito a plazo fijo ofrecido por una entidad bancaria Portuguesa.

Número de registros: 42211

Número de variables: 16, más 1 variable adicional de salida.

Path del Dataset: https://www.kaggle.com/datasets/thedevastator/bank-term-deposit-predictions/

Github: https://github.com/vchirinosb/marketing-campaign/blob/main/marketing-campaign-decoders.ipynb

## I. Decoders:

### Investigación y Comparación de Modelos

1. T5-Small (Text-To-Text Transfer Transformer)
   - Arquitectura: Encoder-Decoder.
   - Tamaño: 60 millones de parámetros.
   - Uso: Versátil para tareas de NLP como traducción, resumen, clasificación, etc.
   - Ventajas: Buena capacidad para múltiples tareas de NLP con un tamaño relativamente pequeño.
   - Desventajas: Podría no ser tan potente como otros modelos más grandes para tareas específicas.

2. GPT-2 (Generative Pre-trained Transformer 2)
   - Arquitectura: Decoder-only.
   - Tamaño: Variantes de 117M, 345M, 762M y 1.5B parámetros.
   - Uso: Generación de texto, conversación, etc.
   - Ventajas: Excelente para generación de texto continuo y coherente.
   - Desventajas: Puede ser costoso en términos de recursos computacionales para variantes más grandes.

3. Pegasus-XSUM
   - Arquitectura: Encoder-Decoder.
   - Tamaño: Aproximadamente 568 millones de parámetros.
   - Uso: Optimizado para tareas de resumen de texto.
   - Ventajas: Muy efectivo en la generación de resúmenes concisos y relevantes.
   - Desventajas: Enfoque especializado puede limitar su rendimiento en otras tareas.
4. LLaMA 3.1
   - Arquitectura_ Basada en la arquitectura Transformer. Diseñada para tareas de procesamiento de lenguaje natural, como clasificación, regresión y generación de texto.
   - Tamaño: 8 mil millones de parámetros.
   - Uso: Predicción de variables de salida binarias, como la aceptación de un depósito a plazo fijo. Análisis de sentimiento y clasificación de texto. Generación de texto y resúmenes.
   - Ventajas: Alta precisión en tareas de clasificación y regresión. Capacidad para manejar datos tabulares y de texto. Robustez ante datos faltantes o inconsistentes. Buen rendimiento en tareas de procesamiento de lenguaje natural.
   - Desventajas: Requiere grandes cantidades de memoria y recursos computacionales.


### Evaluación de los Requisitos del Proyecto:

- Tarea principal: Predicción de aceptación de depósitos.
- Tareas secundarias: Clasificación y resúmenes.


### Proceso de Evaluación y Selección del Modelo:

- Preparación de Datos: Cargar y explorar el dataset.
- Preprocesar datos para que sean compatibles con el modelo.
- Implementación de Modelos
- Comparar resultados y seleccionar el modelo más adecuado.


Justificación de la Elección:

  Para la tarea específica de predicción de aceptación de depósitos a plazo fijo.
- T5-Small: Podría ser una buena opción si los recursos son limitados y se requiere un enfoque multitarea.
- GPT-2: Ideal para generación de texto pero puede no ser la mejor opción para clasificación directa sin modificaciones adicionales.
- Pegasus-XSUM: Especializado en resumen, podría no ser adecuado para clasificación.
- LLaMA 3.1: Potencialmente muy efectivo pero puede requerir recursos significativos.

Basado en estos factores, T5-Small se presenta como una opción balanceada en términos de tamaño y capacidad para tareas de NLP variadas, incluyendo clasificación.

## II. Implementación de Decoders:

### 1. Evaluación usando google/pegasus-xsum

El modelo PEGASUS (Pretrained Generative Attention-based Sequence-to-Sequence) es un modelo de generación de texto.
Diseñado para tareas de resumen de texto y generación de resúmenes coherentes y concisos.

Se usa el modelo PEGASUS-XSum para generar resúmenes a partir de descripciones detalladas.

Modelo: "google/pegasus-xsum"

El código carga datos de clientes, crea descripciones a partir de esos datos, y luego usa el modelo PEGASUS para resumir una de las descripciones, guardando el resultado en un archivo de texto.

Output: El resumen esperado no es el adecuado. Por lo cual el modelo no será seleccionado para entrenamiento.

In [52]:
import pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import time


file_path = 'train.csv'
df = pd.read_csv(file_path)

def create_description(row):
    """
    Crea una descripción concisa basada en los datos de una fila del DataFrame.

    Args:
        row (pd.Series): Fila del DataFrame que contiene datos del cliente.

    Returns:
        str: Descripción concisa del cliente basada en los datos proporcionados.
    """
    try:
        description = (
            f"Client age {row['age']}, job {row['job']}, marital status "
            f"{row['marital']}, education {row['education']}. Balance "
            f"{row['balance']} EUR, housing {row['housing']}, loan "
            f"{row['loan']}. Contacted via {row['contact']} on {row['day']} "
            f"{row['month']}. Last contact duration {row['duration']} sec, "
            f"campaign {row['campaign']} contacts, days since last campaign "
            f"{row['pdays']}, previous outcome {row['poutcome']}. "
            f"Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error al crear la descripción para la fila: {row}\nError: {e}")
        return "Descripción no disponible"

# Aplicar la función para crear la columna 'description'
df['description'] = df.apply(create_description, axis=1)

# Seleccionar la primera descripción para propósitos de prueba.
first_description = df['description'].iloc[0]
print(f"Primera descripción: {first_description}")

try:
    # Cargar el tokenizador y el modelo PEGASUS-XSum.
    model_name = "google/pegasus-xsum"
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)
    print("Modelo y tokenizador cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo/tokenizador: {e}")

try:
    # Tokenizar el texto
    start_time = time.time()
    batch = tokenizer(
        [first_description], max_length=1024, truncation=True,
        padding='longest', return_tensors="pt"
    )
    tokenization_time = time.time() - start_time

    # Generar el resumen usando el modelo PEGASUS
    start_time = time.time()
    translated = model.generate(**batch)
    generation_time = time.time() - start_time

    # Decodificar la secuencia generada
    summary = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

    print(f"Tiempo de tokenización: {tokenization_time:.4f} segundos")
    print(f"Tiempo de generación del resumen: {generation_time:.4f} segundos")
    print(f"Tiempo total: {tokenization_time + generation_time:.4f} segundos")
    print(f"Resumen generado: {summary}")

    # Guardar el resultado en ummaries_pegasus_xsum.txt
    with open('summaries_pegasus_xsum.txt', 'w') as f:
        f.write(f"Descripción Original:\n{first_description}\n")
        f.write(f"Resumen Generado:\n{summary}\n\n")

    print("El resumen ha sido guardado en summaries_pegasus_xsum.txt")
except Exception as e:
    print(f"Error durante la generación del resumen o al guardar: {e}")    

Primera descripción: Client age 58, job management, marital status married, education tertiary. Balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous outcome unknown. Subscription status no.


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo y tokenizador cargados exitosamente.
Tiempo de tokenización: 0.0007 segundos
Tiempo de generación del resumen: 7.3650 segundos
Tiempo total: 7.3657 segundos
Resumen generado: Contact details:
El resumen ha sido guardado en summaries_pegasus_xsum.txt


### 2. Evaluación usando T5-Small

T5-small es una versión reducida del modelo T5, que sigue la arquitectura Transformer y trata todas las tareas de NLP como tareas de traducción de texto a texto.

Uso en el Código: El modelo t5-small se utiliza para resumir descripciones largas generadas a partir de datos de clientes.

Output: El modelo proporciona un resumen correcto, no el óptimo. El modelo será seleccionado para entrenamiento.

In [53]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import time


# Cargar el dataset desde el archivo
file_path = 'train.csv'
df = pd.read_csv(file_path)

def create_description(row):
    """
    Crea una descripción concisa basada en la información de la fila del
    DataFrame.

    Args:
        row (pd.Series): Fila del DataFrame con información del cliente.

    Returns:
        str: Descripción del cliente o mensaje de error si ocurre una excepción.
    """
    try:
        description = (
            f"Client ID {row.name}. Age {row['age']}, job {row['job']}, "
            f"marital status {row['marital']}, education {row['education']}. "
            f"Default status {row['default']}, balance {row['balance']} EUR, "
            f"housing {row['housing']}, loan {row['loan']}. "
            f"Contacted via {row['contact']} on {row['day']} {row['month']}. "
            f"Last contact duration {row['duration']} sec, campaign "
            f"{row['campaign']} contacts, days since last campaign "
            f"{row['pdays']}, previous {row['previous']} contacts, previous "
            f"outcome {row['poutcome']}. Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

# Aplicar la función para crear la columna 'description'
df['description'] = df.apply(create_description, axis=1)

# Seleccionar la primera descripción para fines de prueba.
first_description = df['description'].iloc[0]
print(f"First description: {first_description}")

try:
    # Cargar el tokenizador y el modelo T5.
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model/tokenizer: {e}")

try:
    # Preparar el texto de entrada para el modelo T5.
    input_text = "summarize: " + first_description
    input_ids = tokenizer.encode(
        input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generar el resumen usando el modelo T5.
    start_time = time.time()
    summary_ids = model.generate(
        input_ids, max_length=150, min_length=30,
        length_penalty=2.0, num_beams=4, early_stopping=True)
    generation_time = time.time() - start_time

    # Decodificar la secuencia generada.
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Summary generation time: {generation_time:.4f} seconds")
    print(f"Generated Summary: {summary}")

    # Guardar el resultado en summaries_t5_small.txt
    with open('summaries_t5_small.txt', 'w') as f:
        f.write(f"Original Description:\n{first_description}\n")
        f.write(f"Generated Summary:\n{summary}\n\n")

    print("Summary has been saved to ummaries_t5_small.txt")
except Exception as e:
    print(f"Error during summary generation or saving: {e}")


First description: Client ID 0. Age 58, job management, marital status married, education tertiary. Default status no, balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous 0 contacts, previous outcome unknown. Subscription status no.
Model and tokenizer loaded successfully.
Summary generation time: 2.2947 seconds
Generated Summary: Default status no, balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous 0 contacts, previous outcome unknown.
Summary has been saved to ummaries_t5_small.txt


### 3. Evaluación usando T5-Base

T5-Base es una versión de tamaño intermedio del modelo T5.

Transforma tareas de generación de texto, como el resumen, en un problema de generación de texto a partir de una entrada dada, simplificando el proceso de creación de resúmenes automáticos o respuestas a preguntas.

Output: La salida obtenida de T5_base es similar a T5-small. Por lo que se optará por usar T5-small para entrenamiento.

In [54]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import time


# Cargar el conjunto de datos desde el archivo.
file_path = 'train.csv'
df = pd.read_csv(file_path)

def create_description(row):
    """
    Crear una descripción concisa a partir de una fila del DataFrame.

    Args:
        row (pd.Series): Una fila del DataFrame que contiene datos del cliente.

    Returns:
        str: Una descripción concatenada del cliente o un mensaje de error.
    """
    try:
        description = (
            f"Client age {row['age']}, job {row['job']}, marital status "
            f"{row['marital']}, education {row['education']}. Balance "
            f"{row['balance']} EUR, housing {row['housing']}, loan "
            f"{row['loan']}. Contacted via {row['contact']} on {row['day']} "
            f"{row['month']}. Last contact duration {row['duration']} sec, "
            f"campaign {row['campaign']} contacts, days since last campaign "
            f"{row['pdays']}, previous outcome {row['poutcome']}. "
            f"Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

# Aplicar la función para crear la columna 'description'.
df['description'] = df.apply(create_description, axis=1)

# Seleccionar la primera descripción para fines de prueba.
first_description = df['description'].iloc[0]
print(f"First description: {first_description}")

try:
    # Cargar el tokenizador y el modelo T5.
    model_name = "t5-base"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model/tokenizer: {e}")

try:
    # Preparar el texto de entrada para el modelo T5.
    input_text = "summarize: " + first_description
    input_ids = tokenizer.encode(
        input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generar el resumen usando el modelo T5.
    start_time = time.time()
    summary_ids = model.generate(
        input_ids, max_length=150, min_length=30, length_penalty=2.0,
        num_beams=4, early_stopping=True
    )
    generation_time = time.time() - start_time

    # Decodificar la secuencia generada.
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Summary generation time: {generation_time:.4f} seconds")
    print(f"Generated Summary: {summary}")

    # Guardar el resultado en summaries_t5_base.txt.
    with open('summaries_t5_base.txt', 'w') as f:
        f.write(f"Original Description:\n{first_description}\n")
        f.write(f"Generated Summary:\n{summary}\n\n")

    print("Summary has been saved to summaries_t5_base.txt")
except Exception as e:
    print(f"Error during summary generation or saving: {e}")


First description: Client age 58, job management, marital status married, education tertiary. Balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous outcome unknown. Subscription status no.
Model and tokenizer loaded successfully.
Summary generation time: 4.6403 seconds
Generated Summary: client age 58, job management, marital status married, education tertiary. Contacted via unknown on 5 may.
Summary has been saved to summaries_t5_base.txt


### 4. Evaluación usando GPT2

GPT-2 (Generative Pre-trained Transformer 2) es un modelo de lenguaje desarrollado por OpenAI. Es un modelo basado en la arquitectura de Transformer, diseñado para generar texto de manera coherente y contextualmente relevante

El código genera descripciones textuales para cada entrada, utiliza GPT-2 para crear un resumen de una descripción específica, y guarda tanto la descripción original como el resumen en un archivo para su posterior revisión.

Output: El modelo proporciona un resumen correcto, no el óptimo. El modelo será seleccionado para entrenamiento.

In [57]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import time


# Cargar el conjunto de datos desde el archivo.
file_path = 'train.csv'
df = pd.read_csv(file_path)

# Define a function to create concise descriptions
def create_description(row):
    try:
        description = (
            f"Client ID {row.name}. Age {row['age']}, job {row['job']}, "
            f"marital status {row['marital']}, education {row['education']}. "
            f"Default status {row['default']}, balance {row['balance']} EUR, "
            f"housing {row['housing']}, loan {row['loan']}. Contacted via "
            f"{row['contact']} on {row['day']} {row['month']}. "
            f"Last contact duration {row['duration']} sec, campaign "
            f"{row['campaign']} contacts, days since last campaign "
            f"{row['pdays']}, previous {row['previous']} contacts, "
            f"previous outcome {row['poutcome']}. "
            f"Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

# Aplicar la función para crear la columna 'description'.
df['description'] = df.apply(create_description, axis=1)

# Seleccionar la primera descripción para fines de prueba.
first_description = df['description'].iloc[0]
print(f"First description: {first_description}")

try:
    # Cargar el tokenizador y modelo GPT-2.
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model/tokenizer: {e}")

try:
    # Preparar el texto de entrada para el modelo GPT-2.
    input_text = "Summarize this description: " + first_description
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generar resumen utilizando el modelo GPT-2.
    start_time = time.time()
    summary_ids = model.generate(
        input_ids, max_length=150, min_length=30, length_penalty=2.0,
        num_beams=4, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    generation_time = time.time() - start_time

    # Decodificar la secuencia generada.
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Summary generation time: {generation_time:.4f} seconds")
    print(f"Generated Summary: {summary}")

    # Guardar los resultados en summaries_gpt2.txt
    with open('summaries_gpt2.txt', 'w') as f:
        f.write(f"Original Description:\n{first_description}\n")
        f.write(f"Generated Summary:\n{summary}\n\n")

    print("Summary has been saved to summaries_gpt2.txt")
except Exception as e:
    print(f"Error during summary generation or saving: {e}")


First description: Client ID 0. Age 58, job management, marital status married, education tertiary. Default status no, balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous 0 contacts, previous outcome unknown. Subscription status no.
Model and tokenizer loaded successfully.
Summary generation time: 7.1035 seconds
Generated Summary: Summarize this description: Client ID 0. Age 58, job management, marital status married, education tertiary. Default status no, balance 2143 EUR, housing yes, loan no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous 0 contacts, previous outcome unknown. Subscription status no. Contacted via unknown on 5 may. Last contact duration 261 sec, campaign 1 contacts, days since last campaign -1, previous 0 contacts, previous outcome unknown. Subscription status no. Contacted via unknown

### 5. Evaluación usando Ollama - llama3.1:8b


En el código, LLaMA se utiliza para generar un resumen basado en las descripciones concisas de las filas del conjunto de datos. El modelo recibe el texto combinado como entrada y genera un resumen coherente y relevante. La funcionalidad de LLaMA en este caso es:
- Análisis del texto: LLaMA analiza el texto combinado para entender el contenido y el contexto.
- Generación del resumen: LLaMA genera un resumen coherente y relevante basado en el análisis del texto.

En resumen, el código utiliza LLaMA para generar un resumen basado en descripciones concisas de filas de un conjunto de datos, lo que permite obtener una visión general coherente y relevante del contenido del conjunto de datos.

Output: La respuesta generada es de utilidad. Es un modelo de decoder adecuado para la aplicación al proyecto.

In [59]:
import ollama
import pandas as pd


# Cargar el conjunto de datos desde el archivo.
file_path = 'train.csv'
df = pd.read_csv(file_path)

# Definir una función para crear descripciones concisas.
def create_description(row):
    try:
        description = (
            f"Client ID {row.name}. Age {row['age']}, job {row['job']}, "
            f"marital status {row['marital']}, education {row['education']}. "
            f"Default status {row['default']}, balance {row['balance']} EUR, "
            f"housing {row['housing']}, loan {row['loan']}. Contacted via "
            f"{row['contact']} on {row['day']} {row['month']}. "
            f"Last contact duration {row['duration']} sec, campaign "
            f"{row['campaign']} contacts, days since last campaign "
            f"{row['pdays']}, previous {row['previous']} contacts, previous "
            f"outcome {row['poutcome']}. Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

# Aplica la función para crear la columna 'descripción.
df['description'] = df.apply(create_description, axis=1)

# Combinar las descripciones en un solo texto.
combined_description = " ".join(df['description'])

# Cargar el modelo ollama.
modelo = 'llama3.1:8b'
ollama.pull(modelo)

# Definir el prompt.
prompt = (
    f"Assess the descriptions and predict the probability of subscription: "
    f"{combined_description}"
)

# Generar resumen usando el modelo ollama
response = ollama.generate(model=modelo, prompt=prompt)
print(f"Generated Summary: {response['response']}")

# Guardar el resultado en summaries_ollama.txt
with open('summaries_ollama.txt', 'w') as f:
    f.write(f"Original Descriptions:\n{combined_description}\n")
    f.write(f"Generated Summary:\n{response['response']}\n\n")

print("Summary has been saved to summaries_ollama.txt")

Generated Summary: This appears to be a dataset from a collection agency or debt collector's database. Each entry represents an individual client with their demographic information, credit history, and details about recent contact attempts.

Here are some insights that can be gleaned from this data:

1. **Geographic distribution**: Although not explicitly mentioned in the dataset, the age ranges suggest a Western European country (e.g., Germany, France) as the clients are mostly between 20-80 years old.
2. **Contact methods**: The majority of contacts were made via cellular phone (around 85%); telephone was used for only one client (Client ID 45209).
3. **Age and employment**: Clients span a wide age range, but those in their mid-to-late 30s (35-39 years old) seem to be overrepresented (Clients 45198, 45201). The most common occupations are management (twice), blue-collar, technician, and retired.
4. **Subscription status**: About half of the clients have a positive subscription status

## III. Entrenamiento de los modelos seleccionados:

### 1. Entrenamiento para T5-small:

El modelo T5 (Text-to-Text Transfer Transformer) es un modelo de lenguaje desarrollado por Google Research. La versión "small" de T5 es una variante más compacta del modelo original T5, diseñada para hacer que el modelo sea más accesible en términos de recursos computacionales y tiempos de entrenamiento.

El código está configurado para usar el modelo T5-small en una tarea de clasificación de texto, donde se genera una descripción de entrada y se clasifica si el cliente acepta una oferta o no.

Muestra de Datos: Se usa una muestra del 25% del dataset para entrenamiento, con el objetivo de reducir el tiempo de entrenamiento

División del Dataset: El dataset se divide en conjuntos de entrenamiento y validación. El 80% se usa para entrenamiento y el 20% para validación.

Épocas: Se entrena por 1 época (puedes aumentar esto para un entrenamiento más extensivo).

El modelo y el tokenizador se guardan en el directorio ./saved_t5_model y ./saved_t5_tokenizer, respectivamente.

Output: El modelo de entrenamiento cumple con su objetivo.

In [36]:
import pandas as pd
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import torch
from torch.utils.data import Dataset, random_split
import os


class CustomDataset(Dataset):
    """
    Dataset personalizado para la tokenización de entradas y etiquetas.

    Args:
        encodings (dict): Diccionario con las codificaciones de entrada.
        labels (tensor): Tensores con las etiquetas.

    Métodos:
        __getitem__(idx): Devuelve un ítem en el índice especificado.
        __len__(): Devuelve la longitud del dataset.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach() for key, val in 
            self.encodings.items()
        }
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

def create_description(row):
    """
    Crea una descripción detallada del cliente a partir de una fila del
    DataFrame.

    Args:
        row (pd.Series): Fila del DataFrame con la información del cliente.

    Returns:
        str: Descripción detallada del cliente.
    """
    try:
        description = (
            f"Client ID {row.name}. Age {row['age']}, job {row['job']}, "
            f"marital status {row['marital']}, education {row['education']}. "
            f"Default status {row['default']}, balance {row['balance']} EUR, "
            f"housing {row['housing']}, loan {row['loan']}. Contacted via "
            f"{row['contact']} on {row['day']} {row['month']}. Last contact "
            f"duration {row['duration']} sec, campaign {row['campaign']} "
            f"contacts, days since last campaign {row['pdays']}, previous "
            f"{row['previous']} contacts, previous outcome {row['poutcome']}. "
            f"Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

if not os.path.exists('./saved_t5_model'):
    df = pd.read_csv('train.csv')
    df['description'] = df.apply(create_description, axis=1)

    # Usar solo el 25% del dataset para entrenamiento.
    df_sampled = df.sample(frac=0.25, random_state=42)
    
    text_data = df_sampled['description'].tolist()
    labels = df_sampled['y'].apply(
        lambda x: "yes" if x == 'yes' else "no").tolist()

    model_name = 't5-small'
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    inputs = tokenizer(
        text_data, return_tensors='pt', max_length=512, padding='max_length',
        truncation=True)
    labels = tokenizer(
        labels, return_tensors='pt', max_length=512, padding='max_length',
        truncation=True)['input_ids']

    dataset = CustomDataset(inputs, labels)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=1,  # Reducido a 1 época para ahorrar tiempo.
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,  # Reducido el número de pasos de calentamiento.
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy='steps',
        eval_steps=50,  # Evaluar con más frecuencia.
        save_total_limit=1,  # Mantener solo el mejor modelo.
        save_steps=100,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        # Parada temprana.
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )
    trainer.train()
    model.save_pretrained('./saved_t5_model')
    tokenizer.save_pretrained('./saved_t5_tokenizer')
else:
    model = T5ForConditionalGeneration.from_pretrained('./saved_t5_model')
    tokenizer = T5Tokenizer.from_pretrained('./saved_t5_tokenizer')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

def generate_text(input_text):
    """
    Genera texto utilizando el modelo T5 a partir de una entrada dada.

    Args:
        input_text (str): Texto de entrada para la generación.

    Returns:
        str: Texto generado por el modelo.
    """
    model.eval()
    encoding = tokenizer.encode_plus(
        input_text, return_tensors='pt', max_length=512, padding='max_length',
        truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    output = model.generate(
        input_ids, attention_mask=attention_mask, max_length=150)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [37]:
ejemplos = [
    ("Client ID 1. Age 35, job management, marital status married, education "
     "tertiary. Default status no, balance 1000 EUR, housing yes, loan no. "
     "Contacted via cellular on 5 may. Last contact duration 300 sec, campaign "
     "1 contacts, days since last campaign 999, previous 0 contacts, previous "
     "outcome unknown. Subscription status yes."),
    ("Client ID 2. Age 25, job engineer, marital status single, education "
     "university. Default status no, balance 1500 EUR, housing yes, loan yes. "
     "Contacted via cellular on 10 june. Last contact duration 200 sec, "
     "campaign 2 contacts, days since last campaign 999, previous 1 contacts, "
     "previous outcome failure. Subscription status no."),
    ("Client ID 3. Age 40, job doctor, marital status divorced, education "
     "postgraduate. Default status no, balance 2000 EUR, housing no, loan no. "
     "Contacted via telephone on 15 july. Last contact duration 100 sec, "
     "campaign 3 contacts, days since last campaign 999, previous 2 contacts, "
     "previous outcome success. Subscription status yes."),
]

for ejemplo in ejemplos:
    generated_text = generate_text(ejemplo)
    print(f'Input: {ejemplo}')
    print(f'Output: {generated_text}')
    print('---')

Input: Client ID 1. Age 35, job management, marital status married, education tertiary. Default status no, balance 1000 EUR, housing yes, loan no. Contacted via cellular on 5 may. Last contact duration 300 sec, campaign 1 contacts, days since last campaign 999, previous 0 contacts, previous outcome unknown. Subscription status yes.
Output: yes
---
Input: Client ID 2. Age 25, job engineer, marital status single, education university. Default status no, balance 1500 EUR, housing yes, loan yes. Contacted via cellular on 10 june. Last contact duration 200 sec, campaign 2 contacts, days since last campaign 999, previous 1 contacts, previous outcome failure. Subscription status no.
Output: no
---
Input: Client ID 3. Age 40, job doctor, marital status divorced, education postgraduate. Default status no, balance 2000 EUR, housing no, loan no. Contacted via telephone on 15 july. Last contact duration 100 sec, campaign 3 contacts, days since last campaign 999, previous 2 contacts, previous outco

### 1. Entrenamiento para DISTILGPT2:

DistilGPT-2 es una versión más ligera y eficiente del modelo GPT-2 desarrollado por OpenAI.

Los modelos GPT2 y DistilGPT-2, no pueden ser entrenados localmente, a pesar de haber reducido al minimo los settings de configuración, debido a que se requiere recursos computacionales y GPU.

Se adjunta a continuación el codigo de error obtenido:

RuntimeError: MPS backend out of memory (MPS allocated: 13.45 GB, other allocations: 148.70 MB, max allowed: 13.57 GB). Tried to allocate 147.24 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
import pandas as pd
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import torch
from torch.utils.data import Dataset, random_split
from transformers import DataCollatorForLanguageModeling

class CustomDataset(Dataset):
    """
    Dataset personalizado para la tokenización de entradas y etiquetas.
    """
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach() for key, val in
            self.encodings.items()
        }
        item['labels'] = item['input_ids'].clone().detach()
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

def create_description(row):
    """
    Crea una descripción detallada del cliente a partir de una fila del
    DataFrame.
    """
    try:
        description = (
            f"Client ID {row.name}. Age {row['age']}, job {row['job']}, "
            f"marital status {row['marital']}, education {row['education']}. "
            f"Default status {row['default']}, balance {row['balance']} EUR, "
            f"housing {row['housing']}, loan {row['loan']}. Contacted via "
            f"{row['contact']} on {row['day']} {row['month']}. Last contact "
            f"duration {row['duration']} sec, campaign {row['campaign']} "
            f"contacts, days since last campaign {row['pdays']}, previous "
            f"{row['previous']} contacts, previous outcome {row['poutcome']}. "
            f"Subscription status {row['y']}."
        )
        return description
    except Exception as e:
        print(f"Error creating description for row: {row}\nError: {e}")
        return "Description unavailable"

# Preparar datos.
df = pd.read_csv('train.csv')
df['description'] = df.apply(create_description, axis=1)

# Usar solo el 5% del dataset para entrenamiento inicial.
df_sampled = df.sample(frac=0.05, random_state=42)

text_data = df_sampled['description'].tolist()

model_name = 'distilgpt2'  # Usar un modelo más pequeño.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Agregar un nuevo token de padding.
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# Tokenizar datos.
inputs = tokenizer(
    text_data, return_tensors='pt', max_length=64, padding='max_length',
    truncation=True)

dataset = CustomDataset(inputs)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

training_args = TrainingArguments(
    output_dir='./results_gpt2',
    num_train_epochs=1,  # Reducido a 1 época para ahorrar tiempo
    per_device_train_batch_size=1,  # Reducido el tamaño de batch a 1
    per_device_eval_batch_size=1,  # Reducido el tamaño de batch a 1
    warmup_steps=50,  # Reducido el número de pasos de calentamiento
    weight_decay=0.01,
    logging_dir='./logs_gpt2',
    logging_steps=10,
    eval_steps=50,  # Evaluar con más frecuencia
    save_total_limit=1,  # Mantener solo el mejor modelo.
    save_steps=50,  # Guardar modelo cada 50 pasos.
    load_best_model_at_end=True,  # Necesario para EarlyStoppingCallback.
    eval_strategy="steps",  # Estrategia de evaluación por pasos
)

# Usar DataCollator para manejar datos dinámicamente.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, pad_to_multiple_of=64
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Liberar memoria no utilizada.
if torch.cuda.is_available():
    torch.cuda.empty_cache()
elif torch.backends.mps.is_available():
    torch.mps.empty_cache()

# Entrenar.
trainer.train()

# Guardar modelo.
model.save_pretrained('./saved_gpt2_model')
tokenizer.save_pretrained('./saved_gpt2_tokenizer')

def generate_text(input_text):
    """
    Genera texto utilizando el modelo GPT-2 a partir de una entrada dada.
    """
    model.eval()
    encoding = tokenizer.encode_plus(
        input_text, return_tensors='pt', max_length=64, padding='max_length',  # Reducido max_length a 64
        truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=50)  # Ajustado para respuestas cortas
    return tokenizer.decode(output[0], skip_special_tokens=True)

ejemplos = [
    ("Client ID 1. Age 35, job management, marital status married, education "
     "tertiary. Default status no, balance 1000 EUR, housing yes, loan no. "
     "Contacted via cellular on 5 may. Last contact duration 300 sec, campaign "
     "1 contacts, days since last campaign 999, previous 0 contacts, previous "
     "outcome unknown. Subscription status yes."),
    ("Client ID 2. Age 25, job engineer, marital status single, education "
     "university. Default status no, balance 1500 EUR, housing yes, loan yes. "
     "Contacted via cellular on 10 june. Last contact duration 200 sec, "
     "campaign 2 contacts, days since last campaign 999, previous 1 contacts, "
     "previous outcome failure. Subscription status no."),
    ("Client ID 3. Age 40, job doctor, marital status divorced, education "
     "postgraduate. Default status no, balance 2000 EUR, housing no, loan no. "
     "Contacted via telephone on 15 july. Last contact duration 100 sec, "
     "campaign 3 contacts, days since last campaign 999, previous 2 contacts, "
     "previous outcome success. Subscription status yes."),
]

for ejemplo in ejemplos:
    generated_text = generate_text(ejemplo)
    print(f'Input: {ejemplo}')
    print(f'Output: {generated_text}')
    print('---')